# Classical Shadows with Maestro: PP Scout → MPS Sniper

This notebook demonstrates **entanglement detection using classical shadows** with Maestro's multi-backend pipeline.

The key idea: instead of measuring entanglement everywhere on the lattice (exponentially expensive), we:
1. **Scout** with the Pauli Propagator — a fast Heisenberg-picture method that scans the entire lattice in milliseconds
2. **Snipe** with MPS — run the expensive classical shadows protocol only where entanglement concentrates

---

## Background: Classical Shadows

Full quantum state tomography requires $O(4^n)$ measurements. [Huang et al. (2020)](https://arxiv.org/abs/2002.08953) showed that **classical shadows** can estimate many linear functions of the state with just $O(\log n / \varepsilon^2)$ samples.

A classical shadow snapshot is constructed by:
1. Applying a random single-qubit Clifford $U_i$ to each qubit
2. Measuring in the computational basis → bitstring $\mathbf{b}$
3. Reconstructing: $\hat{\rho}_A = \bigotimes_{q \in A} (3 U_q^\dagger |b_q\rangle\langle b_q| U_q - I)$

The **2nd-order Rényi entropy** is then:
$$S_2 = -\log_2 \text{Tr}(\rho_A^2)$$

which quantifies the entanglement between subsystem $A$ and the rest.

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

import maestro
from maestro.circuits import QuantumCircuit

%matplotlib inline
plt.rcParams['figure.dpi'] = 120

## Step 1: The Physical System — 2D TFIM

We simulate the **transverse-field Ising model** on a 2D square lattice:

$$H = -J \sum_{\langle i,j \rangle} Z_i Z_j - h \sum_i X_i$$

This model exhibits rich entanglement structure — bulk bonds (high coordination) develop more entanglement than corner bonds.

In [ ]:
# Lattice geometry
LX, LY = 4, 4         # 16 qubits (small for exact ED comparison)
N_QUBITS = LX * LY

J_COUPLING = 1.0       # ZZ coupling
H_FIELD = 1.0          # Transverse field
DT = 0.2               # Time step
N_TROTTER = 10         # Trotter steps
CHI = 16               # MPS bond dimension

def site_index(x, y):
    return x * LY + y

def site_coords(idx):
    return idx // LY, idx % LY

def get_nn_bonds():
    bonds = []
    for x in range(LX):
        for y in range(LY):
            idx = site_index(x, y)
            if x + 1 < LX:
                bonds.append((idx, site_index(x + 1, y)))
            if y + 1 < LY:
                bonds.append((idx, site_index(x, y + 1)))
    return bonds

bonds = get_nn_bonds()
print(f"Lattice: {LX}×{LY} = {N_QUBITS} qubits, {len(bonds)} bonds")

In [ ]:
def build_tfim_circuit(n_qubits, bonds, j, h, dt, n_steps):
    """Build TFIM Trotterized time-evolution circuit."""
    qc = QuantumCircuit()
    # Prepare |+⟩^n
    for q in range(n_qubits):
        qc.h(q)
    # Trotter steps
    for _ in range(n_steps):
        for q1, q2 in bonds:
            qc.cx(q1, q2)
            qc.rz(q2, 2.0 * j * dt)
            qc.cx(q1, q2)
        for q in range(n_qubits):
            qc.h(q)
            qc.rz(q, 2.0 * h * dt)
            qc.h(q)
    return qc

print("Circuit builder defined.")

## Step 2: Act 1 — PP Scout

The **Pauli Propagator** works in the Heisenberg picture — tracking operators backward through the circuit. For Clifford gates, a Pauli string maps to a single Pauli string, so the cost is $O(n \cdot d)$ regardless of system size.

We compute $\langle Z_i Z_j \rangle$ for all nearest-neighbor bonds and rank them by a **coordination-weighted score**:

$$\text{score}(i,j) = |\langle Z_i Z_j \rangle| \times \frac{\text{nn}_i + \text{nn}_j}{8}$$

Bulk bonds (high coordination) that are already correlated get the highest score — they will develop the most entanglement.

In [ ]:
def build_pauli_observable(n_qubits, ops):
    """Build a Pauli string: ops is a dict {qubit_idx: 'X'|'Y'|'Z'}."""
    chars = ['I'] * n_qubits
    for q, p in ops.items():
        chars[q] = p
    return ''.join(chars)


def coordination(q):
    """Number of nearest neighbors on the LX×LY lattice."""
    x, y = site_coords(q)
    nn = 0
    if x > 0: nn += 1
    if x < LX - 1: nn += 1
    if y > 0: nn += 1
    if y < LY - 1: nn += 1
    return nn


# Build single-step circuit for PP
scout_circuit = build_tfim_circuit(N_QUBITS, bonds, J_COUPLING, H_FIELD, DT, n_steps=1)

# ZZ observables for all bonds
zz_observables = [
    build_pauli_observable(N_QUBITS, {q1: 'Z', q2: 'Z'})
    for q1, q2 in bonds
]

# Run PP scout
t0 = time.time()
scout_result = scout_circuit.estimate(
    observables=zz_observables,
    simulation_type=maestro.SimulationType.PauliPropagator,
)
scout_time = time.time() - t0

zz_vals = scout_result['expectation_values']
print(f"PP Scout completed in {scout_time:.4f}s")

# Score bonds
scored = []
for (q1, q2), zz in zip(bonds, zz_vals):
    nn_sum = coordination(q1) + coordination(q2)
    score = abs(zz) * nn_sum / 8.0
    scored.append(((q1, q2), zz, score))

scored.sort(key=lambda x: x[2], reverse=True)

hot_qubits = list(scored[0][0])
cold_qubits = list(scored[-1][0])

print(f"\nTop 5 bonds (most entangled):")
for (q1, q2), zz, score in scored[:5]:
    x1, y1 = site_coords(q1)
    x2, y2 = site_coords(q2)
    print(f"  ({q1},{q2}) site ({x1},{y1})↔({x2},{y2})  ⟨ZZ⟩={zz:+.4f}  score={score:.4f}")

print(f"\nHOT subsystem: qubits {hot_qubits}")
print(f"COLD subsystem: qubits {cold_qubits}")

In [ ]:
# Visualize the lattice heatmap
site_score = np.zeros((LX, LY))
for (q1, q2), zz, score in scored:
    x1, y1 = site_coords(q1)
    x2, y2 = site_coords(q2)
    site_score[x1, y1] = max(site_score[x1, y1], score)
    site_score[x2, y2] = max(site_score[x2, y2], score)

fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(site_score, cmap='YlOrRd', aspect='equal')
plt.colorbar(im, ax=ax, label='Scout Score')

# Label sites
for q in range(N_QUBITS):
    x, y = site_coords(q)
    color = 'white' if site_score[x, y] > site_score.max() * 0.5 else 'black'
    ax.text(y, x, str(q), ha='center', va='center', fontsize=9, fontweight='bold', color=color)

# Highlight hot/cold
for label, qubits, color in [('HOT', hot_qubits, 'gold'), ('COLD', cold_qubits, 'deepskyblue')]:
    x1, y1 = site_coords(qubits[0])
    x2, y2 = site_coords(qubits[1])
    ax.plot([y1, y2], [x1, x2], '-', color=color, linewidth=4, zorder=5, label=f'{label}: {qubits}')

ax.legend(loc='upper right', fontsize=9)
ax.set_title(f'PP Scout: {LX}×{LY} TFIM Lattice\nScanned in {scout_time*1000:.1f}ms')
plt.tight_layout()
plt.show()

## Step 3: Act 2 — T-gates Break Clifford → MPS Takeover

The Pauli Propagator handles Clifford gates efficiently because each Pauli maps to a single Pauli: $\text{CX}^\dagger Z \text{CX} = Z$.

**T-gates break this:** $T^\dagger X T = (X + Y)/\sqrt{2}$ — one Pauli becomes **two**. After $k$ T-gates, PP would need to track $2^k$ terms.

Maestro's MPS backend takes over — simulating the full state in the Schrödinger picture with cost controlled by $\chi$.

In [ ]:
# Build circuit with T-gates injected
qc_tgate = build_tfim_circuit(N_QUBITS, bonds, J_COUPLING, H_FIELD, DT, N_TROTTER)
n_t_gates = min(N_QUBITS // 2, 8)
for q in range(n_t_gates):
    qc_tgate.t(q)

print(f"Injected {n_t_gates} T-gates")
print(f"PP would need 2^{n_t_gates} = {2**n_t_gates} Pauli terms")
print(f"MPS handles it with χ={CHI}")

# Compute energy with MPS
energy_obs = []
for q1, q2 in bonds:
    energy_obs.append(build_pauli_observable(N_QUBITS, {q1: 'Z', q2: 'Z'}))
for q in range(N_QUBITS):
    energy_obs.append(build_pauli_observable(N_QUBITS, {q: 'X'}))

t0 = time.time()
mps_result = qc_tgate.estimate(
    observables=energy_obs,
    simulator_type=maestro.SimulatorType.QCSim,
    simulation_type=maestro.SimulationType.MatrixProductState,
    max_bond_dimension=CHI,
)
mps_time = time.time() - t0

exp_vals = mps_result['expectation_values']
n_bonds = len(bonds)
e_zz = sum(-J_COUPLING * exp_vals[i] for i in range(n_bonds))
e_x = sum(-H_FIELD * exp_vals[n_bonds + i] for i in range(N_QUBITS))

print(f"\nEnergy (MPS, χ={CHI}): E = {e_zz + e_x:.4f}")
print(f"Computed in {mps_time:.3f}s")

## Step 4: Classical Shadow Estimation

Now we implement the full classical shadow protocol:

1. Build the TFIM circuit at a given Trotter depth
2. Apply a **random single-qubit Clifford** to each qubit
3. **Measure** (sample 1 bitstring)
4. **Reconstruct** the shadow density matrix: $\hat{\rho}_A = \bigotimes_{q \in A} (3 U_q^\dagger |b_q\rangle\langle b_q| U_q - I)$
5. Repeat M times and estimate $\text{Tr}(\rho_A^2)$ via U-statistics

In [ ]:
# Clifford gates and their matrices
CLIFFORD_GATES = ['I', 'H', 'HS', 'SH', 'HSdg', 'SHSdg']

def apply_clifford_gate(qc, qubit, label):
    if label == 'I': pass
    elif label == 'H': qc.h(qubit)
    elif label == 'HS': qc.s(qubit); qc.h(qubit)
    elif label == 'SH': qc.h(qubit); qc.s(qubit)
    elif label == 'HSdg': qc.sdg(qubit); qc.h(qubit)
    elif label == 'SHSdg': qc.sdg(qubit); qc.h(qubit); qc.s(qubit)

def clifford_matrix(label):
    I = np.eye(2, dtype=complex)
    H = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2)
    S = np.array([[1, 0], [0, 1j]], dtype=complex)
    Sdg = np.array([[1, 0], [0, -1j]], dtype=complex)
    return {'I': I, 'H': H, 'HS': H @ S, 'SH': S @ H, 'HSdg': H @ Sdg, 'SHSdg': S @ H @ Sdg}[label]

def build_shadow_snapshot(bits, labels, subsystem):
    """Build reduced shadow ρ̂_A from a single measurement."""
    shadows = []
    for q in subsystem:
        b = bits[q] if q < len(bits) else 0
        ket = np.array([[1 - b], [b]], dtype=complex)
        proj = ket @ ket.conj().T
        U = clifford_matrix(labels[q])
        shadow_q = 3.0 * (U.conj().T @ proj @ U) - np.eye(2, dtype=complex)
        shadows.append(shadow_q)
    rho = shadows[0]
    for i in range(1, len(shadows)):
        rho = np.kron(rho, shadows[i])
    return rho

print("Shadow reconstruction functions defined.")

In [ ]:
def estimate_purity(shadows):
    """Unbiased U-statistics estimator for Tr(ρ²)."""
    d_A = shadows[0].shape[0]
    running_sum = np.zeros((d_A, d_A), dtype=complex)
    cross_total = 0.0
    for i, rho in enumerate(shadows):
        if i > 0:
            cross_total += np.real(np.trace(rho @ running_sum))
        running_sum += rho
    M = len(shadows)
    return float((2.0 * cross_total) / (M * (M - 1)))


def collect_shadows(n_trotter_steps, subsystem_qubits, n_shadows=200):
    """Collect M shadow snapshots for a given Trotter depth."""
    shadows = []
    for s_idx in range(n_shadows):
        rng = np.random.default_rng(seed=s_idx)
        qc = build_tfim_circuit(N_QUBITS, bonds, J_COUPLING, H_FIELD, DT, n_trotter_steps)
        
        # Random Clifford layer
        labels = []
        for q in range(N_QUBITS):
            label = rng.choice(CLIFFORD_GATES)
            labels.append(label)
            apply_clifford_gate(qc, q, label)
        
        qc.measure_all()
        result = qc.execute(
            simulator_type=maestro.SimulatorType.QCSim,
            simulation_type=maestro.SimulationType.MatrixProductState,
            shots=1,
            max_bond_dimension=CHI,
        )
        bitstring = list(result['counts'].keys())[0]
        bits = [int(b) for b in bitstring[:N_QUBITS]]
        shadows.append(build_shadow_snapshot(bits, labels, subsystem_qubits))
    return shadows

print("Shadow collection pipeline defined.")

## Step 5: Act 6 — Hot vs Cold Entanglement Sweep

Now we run the classical shadows sweep on **both** subsystems identified by the PP scout:
- **HOT** (bulk bond, high coordination) — PP predicts strong entanglement growth
- **COLD** (corner bond, low coordination) — PP predicts weak entanglement

If the scout is correct, the hot subsystem should show significantly higher $S_2$.

In [ ]:
trotter_depths = [1, 2, 3, 4, 6, 8, 10]
n_shadows = 200

print(f"Running shadow sweep on HOT={hot_qubits} and COLD={cold_qubits}")
print(f"Depths: {trotter_depths}, Shadows/depth: {n_shadows}\n")

hot_s2, cold_s2 = [], []
times = []

for depth in trotter_depths:
    t_val = depth * DT
    times.append(t_val)
    
    # Hot subsystem
    hot_shadows = collect_shadows(depth, hot_qubits, n_shadows)
    hot_purity = estimate_purity(hot_shadows)
    hot_purity_clamped = float(np.clip(hot_purity, 0.25, 1.0))
    hot_s2_val = -np.log2(hot_purity_clamped)
    hot_s2.append(hot_s2_val)
    
    # Cold subsystem
    cold_shadows = collect_shadows(depth, cold_qubits, n_shadows)
    cold_purity = estimate_purity(cold_shadows)
    cold_purity_clamped = float(np.clip(cold_purity, 0.25, 1.0))
    cold_s2_val = -np.log2(cold_purity_clamped)
    cold_s2.append(cold_s2_val)
    
    print(f"  depth={depth:2d} (t={t_val:.2f})  HOT S₂={hot_s2_val:.4f}  COLD S₂={cold_s2_val:.4f}")

print("\nSweep complete!")

In [ ]:
# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Hot subsystem
ax1.plot(times, hot_s2, 'o--', color='#7B1FA2', linewidth=2, markersize=7,
         label='Classical shadows (MPS)')
ax1.axhline(y=2.0, color='gray', linestyle=':', alpha=0.5, label='Max S₂ (2 qubits)')
hx0, hy0 = site_coords(hot_qubits[0])
hx1, hy1 = site_coords(hot_qubits[1])
ax1.set_xlabel('Simulation Time t')
ax1.set_ylabel('Rényi Entropy S₂')
ax1.set_title(f'HOT Subsystem (PP-selected)\nQubits {hot_qubits} — site ({hx0},{hy0})↔({hx1},{hy1})')
ax1.legend(fontsize=9)
ax1.grid(alpha=0.3)
ax1.set_ylim(-0.05, 2.15)

# Cold subsystem
ax2.plot(times, cold_s2, 's--', color='#E65100', linewidth=2, markersize=7,
         label='Classical shadows (MPS)')
ax2.axhline(y=2.0, color='gray', linestyle=':', alpha=0.5, label='Max S₂ (2 qubits)')
cx0, cy0 = site_coords(cold_qubits[0])
cx1, cy1 = site_coords(cold_qubits[1])
ax2.set_xlabel('Simulation Time t')
ax2.set_ylabel('Rényi Entropy S₂')
ax2.set_title(f'COLD Subsystem (contrast)\nQubits {cold_qubits} — site ({cx0},{cy0})↔({cx1},{cy1})')
ax2.legend(fontsize=9)
ax2.grid(alpha=0.3)
ax2.set_ylim(-0.05, 2.15)

fig.suptitle(f'Entanglement Growth — {LX}×{LY} TFIM  |  PP Scout → MPS Sniper',
             fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

hot_mean = np.mean(hot_s2)
cold_mean = np.mean(cold_s2)
print(f"\nAverage S₂:  HOT = {hot_mean:.3f}  |  COLD = {cold_mean:.3f}")
if hot_mean > cold_mean:
    print(f"→ PP Scout confirmed: HOT subsystem has more entanglement (ΔS₂ = {hot_mean - cold_mean:.3f})")
else:
    print(f"→ Both subsystems show similar entanglement")

## Step 6: Efficiency Analysis

Let's quantify the efficiency gain from the PP Scout → MPS Sniper approach.

In [ ]:
full_tomography = 4 ** N_QUBITS
shadows_total = n_shadows * len(trotter_depths)

print(f"System: {LX}×{LY} = {N_QUBITS} qubits")
print(f"")
print(f"Full tomography:    4^{N_QUBITS} = {full_tomography:.2e} measurements")
print(f"Classical shadows:  {shadows_total} snapshots total")
print(f"Speedup:            {full_tomography / shadows_total:.1e}×")
print(f"")
print(f"PP Scout time:      {scout_time*1000:.1f} ms (scanned all {len(bonds)} bonds)")
print(f"MPS Sniper:         {n_shadows} snapshots × {len(trotter_depths)} depths")
print(f"")
print("Without the scout, you'd have to run shadows on EVERY possible")
print(f"subsystem = {N_QUBITS * (N_QUBITS - 1) // 2} pairs × {n_shadows} shadows each.")
print(f"Scout reduces this to just 2 targeted subsystems.")

## Summary

| Act | Backend | Purpose | Cost |
|-----|---------|---------|------|
| 1. PP Scout | `PauliPropagator` | Find entanglement hotspots | O(n·d), milliseconds |
| 2. T-gates | — | Break Clifford → force MPS | — |
| 3. Handoff | `MPS` χ_low → χ_high | Adaptive bond dimension | O(χ³) |
| 4. Cliffords | Classical | Tomographic completeness | O(1) per qubit |
| 5. Sampling | `MPS execute()` | Hardware emulation | O(n·χ²) |
| 6. Sniper | `MPS` + NumPy | Shadow estimation on hot/cold | M snapshots |

**Key takeaway:** The Pauli Propagator scouts in milliseconds where statevector would need $2^n$ amplitudes. MPS then runs targeted shadows only where they matter. At 36 qubits, classical shadows need 200 snapshots vs $4^{36} \approx 10^{21}$ for full tomography — a $10^{19}\times$ speedup.